### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [52]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [72]:
importlib.reload(train)

models = {
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50, 70, 80, 200],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 50, 70, 100],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7]
    }],
    "XGB": [{
        "use_label_encoder": [False],
        "objective": ["binary:logistic"],
        "eval_metric": ["error", "mean_absolute_error", "mean_squared_error", "log_loss"]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]
    # + train.Metrics.STATISTICAL

In [73]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.2, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data["best_params"]}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    # features = {name: metrics for name in models.keys()}
    features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=4)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

One or more of the test scores are non-finite: [0.53825863 0.54949062 0.55013654 0.5590952  0.56353116 0.56796182
 0.58887019 0.56783053 0.56048127 0.55555137 0.55957336 0.56725622
 0.57404103 0.5707421  0.55927996 0.57913558 0.57989981 0.57529595
 0.58546356 0.59327934 0.57862346 0.57481465 0.58263952 0.58259273
 0.55031058 0.53961434 0.55783579 0.56441057 0.56464307 0.57374362
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.58263094        nan        nan        nan]


Model LGB, Actual: [0 1], [127 116], Predictions: [0 1], [136 107]
Model RF, Actual: [0 1], [127 116], Predictions: [0 1], [147  96]
Model XGB, Actual: [0 1], [127 116], Predictions: [0 1], [142 101]

[('bpm', 247), ('rmssd', 230), ('hf_rr', 226), ('lf_rr', 190)]


[('hf_rr', 0.2590566667800719), ('rmssd', 0.2555615058143746), ('lf_rr', 0.2482723572376253), ('bpm', 0.2371094701679283)]


[('hf_rr', 0.28438595), ('rmssd', 0.24708931), ('lf_rr', 0.2392628), ('bpm', 0.229262)]



One or more of the test scores are non-finite: [0.53936811 0.53929709 0.5564545  0.58112059 0.57933907 0.58015256
 0.58045125 0.5719008  0.58131185 0.58701289 0.57679766 0.57921483
 0.57376171 0.57874824 0.56925021 0.57461935 0.57823293 0.58324104
 0.54568281 0.56442081 0.56510095 0.57186106 0.56417367 0.56298385
 0.53759314 0.5765438  0.57264013 0.58511493 0.5723368  0.57761519
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.51958377        nan        nan        nan]


Model LGB, Actual: [0 1], [142  84], Predictions: [0 1], [157  69]
Model RF, Actual: [0 1], [142  84], Predictions: [0 1], [136  90]
Model XGB, Actual: [0 1], [142  84], Predictions: [0 1], [142  84]

[('bpm', 158), ('hf_rr', 128), ('rmssd', 120), ('lf_rr', 104)]


[('rmssd', 0.2674875216960928), ('bpm', 0.24739786639405317), ('lf_rr', 0.24570267164735501), ('hf_rr', 0.23941194026249918)]


[('bpm', 0.28271165), ('lf_rr', 0.26904765), ('hf_rr', 0.23693343), ('rmssd', 0.21130736)]



One or more of the test scores are non-finite: [0.56122145 0.58088111 0.59494565 0.58346813 0.59097742 0.59424016
 0.56733268 0.59408013 0.60205941 0.59222209 0.59757474 0.59503784
 0.60742142 0.61132786 0.60031935 0.60596039 0.59596644 0.59654428
 0.57906665 0.58955682 0.59493606 0.59735328 0.589261   0.58882907
 0.54826763 0.55257177 0.57786868 0.60256463 0.60042668 0.59674599
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.58998491        nan        nan        nan]


Model LGB, Actual: [0 1], [135  61], Predictions: [0 1], [127  69]
Model RF, Actual: [0 1], [135  61], Predictions: [0 1], [123  73]
Model XGB, Actual: [0 1], [135  61], Predictions: [0 1], [ 96 100]

[('hf_rr', 425), ('bpm', 361), ('rmssd', 330), ('lf_rr', 319)]


[('bpm', 0.28235579536806854), ('rmssd', 0.2463745833999858), ('lf_rr', 0.24137799728140602), ('hf_rr', 0.22989162395053953)]


[('hf_rr', 0.2731189), ('lf_rr', 0.2672981), ('bpm', 0.24992332), ('rmssd', 0.20965964)]



One or more of the test scores are non-finite: [0.53442649 0.57279234 0.58473725 0.59401861 0.59444057 0.58826379
 0.54794967 0.55441142 0.56459021 0.56843248 0.56728568 0.57748004
 0.55296244 0.56101238 0.57960719 0.57870176 0.58532154 0.58644114
 0.57284686 0.58041917 0.58665352 0.57908441 0.58947858 0.58707174
 0.57818529 0.60517948 0.59617392 0.59919193 0.60303875 0.60166392
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.60523394        nan        nan        nan]


Model LGB, Actual: [0 1], [136  95], Predictions: [0 1], [148  83]
Model RF, Actual: [0 1], [136  95], Predictions: [0 1], [132  99]
Model XGB, Actual: [0 1], [136  95], Predictions: [0 1], [131 100]

[('bpm', 142), ('hf_rr', 137), ('lf_rr', 134), ('rmssd', 120)]


[('bpm', 0.29215439929209924), ('rmssd', 0.2680797401174727), ('lf_rr', 0.23128864178904057), ('hf_rr', 0.20847721880138742)]


[('lf_rr', 0.25749797), ('bpm', 0.25310382), ('hf_rr', 0.24480224), ('rmssd', 0.24459597)]



One or more of the test scores are non-finite: [0.52300597 0.56514545 0.5557725  0.57094942 0.57776599 0.57356073
 0.51664448 0.5448993  0.52531346 0.54489235 0.55181754 0.55472092
 0.51086788 0.52495567 0.54244967 0.57062799 0.56354268 0.57242384
 0.57492077 0.61220622 0.59086649 0.58613738 0.58306081 0.5846204
 0.5416777  0.56490168 0.56610969 0.56850229 0.56239343 0.56081423
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.54093086        nan        nan        nan]


Model LGB, Actual: [0 1], [155  76], Predictions: [0 1], [160  71]
Model RF, Actual: [0 1], [155  76], Predictions: [0 1], [145  86]
Model XGB, Actual: [0 1], [155  76], Predictions: [0 1], [133  98]

[('bpm', 152), ('rmssd', 149), ('hf_rr', 141), ('lf_rr', 63)]


[('bpm', 0.3170874171760817), ('hf_rr', 0.247506898257205), ('rmssd', 0.23785668268187296), ('lf_rr', 0.1975490018848402)]


[('hf_rr', 0.27436736), ('rmssd', 0.265129), ('lf_rr', 0.24348755), ('bpm', 0.21701607)]

Model evaluation metrics for LGB:
	Accuracy: 0.5020576131687243
	Precision: 0.4766355140186916
	Recall: 0.4396551724137931
	F1-score: 0.45739910313901344
	AUC score: 0.4993551452620147
----------------------------------------
	Accuracy: 0.5707964601769911
	Precision: 0.4057971014492754
	Recall: 0.3333333333333333
	F1-score: 0.3660130718954248
	AUC score: 0.522300469483568
----------------------------------------
	Accuracy: 0.5612244897959183
	Precision: 0.3188405797101449
	Recall: 0.36065573770491804
	F1-score: 0.3

In [ ]:
# LOAD TRAIN AND TEST DATA -- train on some phases and test on others
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

# metrics = train.Metrics.ALL
# metrics = train.Metrics.ECG \
    # + train.Metrics.EDA \
    # + train.Metrics.ANKLE + train.Metrics.WRIST

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    # train.Metrics.HF_RR, 
    # train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]

phases_a = [
    "Baseline_Rest", 
    "BugBox_Relax", 
    "BugBox_Anticipate", 
    # "BugBox_Exposure", 
    # "BugBox_Break",
    "Speech_Relax", 
    "Speech_Anticipate", 
    # "Speech_Exposure", 
    # "Speech_Break"
]

phases_b = [
    # "Baseline_Rest", 
    # "BugBox_Relax", 
    # "BugBox_Anticipate", 
    "BugBox_Exposure", 
    # "BugBox_Break",
    # "Speech_Relax", 
    # "Speech_Anticipate", 
    "Speech_Exposure", 
    # "Speech_Break"
]

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

anxiety_label_type = "Anxiety"
# anxiety_label_type = None
test_size = 1.0

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, phases_a, verbose=False, anxiety_label_type=anxiety_label_type)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_b, verbose=False, anxiety_label_type=anxiety_label_type)
x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# drop subjects with noisy data

x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)]
y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)]
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)]
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)]

print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

# x = x[x['subject'] != 8.0]
# y = y[y['subject'] != 8.0]

if anxiety_label_type is not None:
    x_a.drop(labels=["anxietyGroup"], axis=1)
    x_b.drop(labels=["anxietyGroup"], axis=1)

# 0-1 scaling
for i in range(3, len(x_a.columns)):
    data_col = x_a[x_a.columns[i]]
    data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
    x_a[x_a.columns[i]] = data_col
for i in range(3, len(x_b.columns)):
    data_col = x_b[x_b.columns[i]]
    data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
    x_b[x_b.columns[i]] = data_col
    
# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LogReg": [],
    "XGB": []
}
reports = {
    "SVM": [],
    "LogReg": [],
    "XGB": []
}
num_iters = 10
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0.4:
        print(f"Model evaluation metrics for {model_name}:")
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
    print("")
print("\n")